In [ ]:
# Note that this notebook is originally written in Google Colab
# This code is for output formatting, so that the text is wrapped (for easier reading)

from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
# Import required packages
# NOTE (IMPORTANT): Unfortunately, the way this works, the notebook needs to be refreshed after this is called once
# So, when you first run the notebook, run this cell --> There will be an error saying that you need to refresh
# Then, restart runtime and run all (everything should work well this time)
!pip install bertopic

In [ ]:
# Mount Gdrive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import os

# Regular modules for data science and visualization:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# Keras (2.2.4) and tensorflow (1.13).
import tensorflow as tf
import tensorflow_hub as hub

from keras.regularizers import l1, l2
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import backend
from keras import optimizers

#sklearn and imblearn modules:
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from imblearn.over_sampling import SMOTE

from sklearn.metrics import confusion_matrix

In [ ]:
root = '/content/drive/My Drive/CS489 Team 3 Project'

In [ ]:
# Load text and dataset
df = pd.read_csv(root + "/Irfan's code/articles3.csv")

In [ ]:
small_df = df[3000:3010]

In [ ]:
#save the array for later usage
e_All_four = np.load(root + "/Irfan's code/e_All_four_0_2.npy")

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
# Load the encoder:
g = tf.Graph()
with g.as_default():
  text_input = tf.placeholder(dtype=tf.string, shape=[None])
  embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-large/3")
  embedded_text = embed(text_input)
  init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

# Initialize session:
session = tf.Session(graph=g)
session.run(init_op)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
scaler = StandardScaler()

In [ ]:
# Load models
from bertopic import BERTopic

bert_topic = BERTopic.load(root + "/Chrysan's code/topic_model")
news_DNN_four = tf.keras.models.load_model(root + "/Irfan's code/news_DNN_four")

In [ ]:
small_df

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
2000,105459,155773,NBA star Dwyane Wade calls Chicago’s gun laws ...,Guardian,NaN,2016-09-02,2016.0,9.0,https://www.theguardian.com/us-news/2016/sep/0...,Dwyane Wade has lashed out against his hometow...
2001,105460,155774,DNA database brings scientists closer to pinpo...,Guardian,Nicola Davis,2016-08-17,2016.0,8.0,https://www.theguardian.com/science/2016/aug/1...,Scientists say they are closer to pinning down...
2002,105461,155775,Five Star mayor of Turin to create Italy’s fir...,Guardian,Stephanie Kirchgaessner,2016-07-21,2016.0,7.0,https://www.theguardian.com/world/2016/jul/21/...,From vitello tonnato — veal with tuna sauce...
2003,105462,155777,FX takes major steps to improve diversity in ’...,Guardian,Nigel M Smith,2016-08-09,2016.0,8.0,https://www.theguardian.com/media/2016/aug/09/...,"FX, the TV network that dominated this year’s ..."
2004,105463,155778,American obsession: how JonBenét Ramsey gave r...,Guardian,Michelle Dean,2016-09-09,2016.0,9.0,https://www.theguardian.com/us-news/2016/sep/0...,"In February 1997, a regular visitor to the pop..."
2005,105464,155779,Rio 2016: Spain beat Australia 89-88 to earn d...,Guardian,NaN,2016-08-21,2016.0,8.0,https://www.theguardian.com/sport/2016/aug/21/...,Spain beat Australia in a thrilling clash to...
2006,105465,155781,Donald Trump still set on relocating US embass...,Guardian,Peter Beaumont,2016-12-12,2016.0,12.0,https://www.theguardian.com/world/2016/dec/12/...,A senior adviser to Donald Trump has reiterate...
2007,105466,155782,Shannon Purser: ‘People have actually had Barb...,Guardian,Michael Hogan,2016-12-17,2016.0,12.0,https://www.theguardian.com/tv-and-radio/2016/...,‘To say 2016 has been a big year for me is an ...
2008,105467,155783,Classmate of 43 missing Mexican students was t...,Guardian,NaN,2016-07-11,2016.0,7.0,https://www.theguardian.com/world/2016/jul/11/...,A classmate of 43 college students who disappe...
2009,105468,155784,"Hey, Jude: Montana welcomes new dinosaur ’Judi...",Guardian,NaN,2016-05-18,2016.0,5.0,https://www.theguardian.com/science/2016/may/1...,A novice fossil collector’s lucky find in a re...


In [ ]:
for i, row in small_df.iterrows():
  topic_num = bert_topic.transform(row['content'])[0][0]
  keywords = bert_topic.get_topic(topic_num)
  lst = [entry[0] for entry in keywords]
  print(lst)

['police', 'officers', 'officer', 'cops', 'policing', 'ferguson', 'violence', 'enforcement', 'blacks', 'departments']
['crispr', 'embryos', 'genes', 'dna', 'scientists', 'genome', 'gene', 'researchers', 'fda', 'patents']
['school', 'family', 'social', 'things', 'children', 'women', 'america', 'story', 'media', 'police']
['school', 'family', 'social', 'things', 'children', 'women', 'america', 'story', 'media', 'police']
['school', 'family', 'social', 'things', 'children', 'women', 'america', 'story', 'media', 'police']
['school', 'family', 'social', 'things', 'children', 'women', 'america', 'story', 'media', 'police']
['israel', 'israeli', 'palestinian', 'netanyahu', 'palestinians', 'israels', 'israelis', 'gaza', 'netanyahus', 'hamas']
['school', 'family', 'social', 'things', 'children', 'women', 'america', 'story', 'media', 'police']
['guzman', 'chapo', 'mexican', 'mexico', 'cartel', 'mexicos', 'extradition', 'guzmans', 'trafficking', 'ciudad']
['dinosaurs', 'fossils', 'species', 'dino

In [ ]:
def polarization_score(pred_list, cls):
  if cls == 0 or cls == 3:
    return int(50 + 50 * pred_list[cls])
  else:
    return int(50 * pred_list[cls])

In [ ]:
def make_pred(news_text):
  sim_matrix = session.run(embedded_text, feed_dict={text_input: [news_text]})
  combine = np.vstack([e_All_four, sim_matrix])
  sca = scaler.fit_transform(combine)[-1]
  sca = np.array([sca])
  # sca = sca.reshape(sca.shape[0], 1, sca.shape[1])
  pred = news_DNN_four.predict(sca)
  classes_y= np.argmax(pred,axis=1)

  # topic modelling addition
  topic_num = bert_topic.transform(news_text)[0][0]
  keywords = bert_topic.get_topic(topic_num)
  keywords_lst = [entry[0] for entry in keywords]

  if classes_y[0] == 0:
    return 'Left', polarization_score(pred[0], classes_y), keywords_lst
  elif  classes_y[0] == 1:
    return 'Left-center', polarization_score(pred[0], classes_y), keywords_lst
  elif  classes_y[0] == 2:
    return 'right-center', polarization_score(pred[0], classes_y), keywords_lst
  else:
    return 'right', polarization_score(pred[0], classes_y), keywords_lst

In [ ]:
for i, row in small_df.iterrows():
  output = make_pred(row['content'])
  print(output)

('Left-center', 22, ['roof', 'roofs', 'jury', 'trial', 'jurors', 'sentencing', 'prosecutors', 'testimony', 'prosecutor', 'judge'])
('Left-center', 26, ['police', 'officers', 'officer', 'cops', 'policing', 'ferguson', 'violence', 'enforcement', 'blacks', 'departments'])
('Left-center', 39, ['dinosaurs', 'fossils', 'species', 'dinosaur', 'neanderthals', 'specimens', 'sapiens', 'evolution', 'evolved', 'tyrannosaurus'])
('right', 74, ['school', 'family', 'social', 'things', 'children', 'women', 'america', 'story', 'media', 'police'])
('right', 78, ['climate', 'agreement', 'emissions', 'treaty', 'carbon', 'coal', 'nations', 'pact', 'kyoto', 'agreements'])
('Left-center', 26, ['pipeline', 'dakota', 'tribe', 'sioux', 'tribes', 'tribal', 'protesters', 'pipelines', 'indigenous', 'lands'])
('right', 86, ['school', 'family', 'social', 'things', 'children', 'women', 'america', 'story', 'media', 'police'])
('Left-center', 31, ['zika', 'virus', 'mosquitoes', 'mosquito', 'infected', 'infection', 'den